# **Modelo de irradiância incidente - funções auxiliares**

Este notebook apresenta funções auxiliares utilizadas no notebook `modelo_irradiancia`.

**Autor:** Bruno Rech | **Criado em:** 2025-01-21

* * *

### Bibliotecas requeridas

In [ ]:
# Importar
import os
import ee
import numpy as np
import pandas as pd

# Inicializar GEE
try:

    ee.Initialize()

except:

    ee.Authenticate()
    ee.Initialize()
from scipy import stats

### Dados de campo

In [ ]:
def openFieldData(path, equipment):

    # Processamento para dados do Satlantic
    if equipment == 'satlantic':
        
        # Carregar dados
        field_data = pd.read_csv(path).dropna()
        
        # Horários de coleta
        times = [f'{x[:2]}:{x[2:4]}' 
                 for x in field_data.time.astype(str)]
        
        # Tranformar em hora (UTC)
        field_data['time'] = pd.to_datetime(times, format='%H:%M')
        
        # Remover colunas sem serventia
        field_data = field_data.drop(columns=['Unnamed: 0',
                                              'depth', 'Data'])
        
        # Considerar mediana dos espectros
        es = field_data.groupby('station_id').median().reset_index()
        
        # Salvar apenas horário
        es['time'] = [x.strftime('%H:%M') for x in es.time]
        
        # Podemos remover a ID do ponto, identificamos pelo horário
        es.drop('station_id', axis=1, inplace=True)
        
        # O horário passa a ser o índice
        es.set_index('time', inplace=True)
        
        # Alterar formato e converter unidade
        es = es.transpose() / 100
        
        # Ajustar comprimento de onda
        es['wl'] = [int(w[1:]) for w in es.index]
        
        # Ajustar índices
        es.reset_index(drop=True, inplace=True)
        es.rename_axis(None, axis=1, inplace=True)

        return es

    elif equipment == 'trios':

        print('Processamento do TriOS em implementação')

    else:

        raise Exception('Indique o equipamento!')

### Parâmetros atmosféricos

In [ ]:
def atmParams(date, td_aod, coords):

    date = pd.to_datetime(date)
    
    # Região de interesse
    roi = ee.Geometry.Rectangle(coords)
    
    # Profundidade óptica de aerossois [adimensional]
    try:
    
        aod_550 = (
            
            ee.ImageCollection('MODIS/061/MCD19A2_GRANULES')
            .filterDate(date - pd.Timedelta(td_aod, 'd'),
                        date + pd.Timedelta(td_aod + 1, 'd'))
            .filterBounds(roi)
            .select('Optical_Depth_055')
            .mean()
            .reduceRegion(reducer=ee.Reducer.mean(), geometry=roi,
                         scale=1000)
            .getNumber('Optical_Depth_055')
            .multiply(0.001)
            .getInfo()
        
        )
    
        print(f'AOD média na área de interesse: {aod_550:.3f}')
    
    except:
    
        print('Sem dados de AOD! Aumente o timedelta e tente novamente. \n')
    
    
    # Ozônio [atm-cm] 
    # O fator de conversão de mol/m² para atm-cm é 2.238
    ozone = (
        
        ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_O3')
        .filterDate(date, date + pd.Timedelta(1, 'd'))
        .filterBounds(roi)
        .select('O3_column_number_density')
        .mean()
        .reduceRegion(reducer=ee.Reducer.mean(), geometry=roi)
        .getNumber('O3_column_number_density')
        .multiply(2.238)
        .getInfo()
    
    )
    
    print(f'Ozônio médio na área de interesse: {ozone:.3f} atm-cm')
    
    # Vapor d'água [cm]
    # Conversão de kg/m² para cm é 1/10
    water = (
    
        ee.ImageCollection('NOAA/CFSR')
        .filterDate(date, date + pd.Timedelta(1, 'd'))
        .filterBounds(roi)
        .select('Precipitable_water_entire_atmosphere_single_layer')
        .mean()
        .reduceRegion(reducer=ee.Reducer.mean(), geometry=roi)
        .getNumber('Precipitable_water_entire_atmosphere_single_layer')
        .divide(10)
        .getInfo()
    
    )
    
    print(f'Água na atmosfera na área de interesse: {water:.2f} cm')
    
    # Pressão [Pa]
    # O dataset se limite ao continente
    pressure = (
        
        ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
        .filterDate(date, date + pd.Timedelta(1, 'd'))
        .filterBounds(roi)
        .select('surface_pressure')
        .mean()
        .reduceRegion(reducer=ee.Reducer.mean(), geometry=roi)
        .getNumber('surface_pressure')
        .getInfo()
    
    )
    
    # Caso não haja dados, o valor padrão é adotado
    if pressure == None:
    
        pressure = 101320
    
    pressure
    print(f'Pressão atmosférica na área de interesse: {pressure:.0f} Pa')

    return aod_550, ozone, water, pressure

### Métricas de erro

In [2]:
def SAM(x, y):

    # Calculate the similarity
    d = np.acos((x*y).sum() / ((x**2).sum()*(y**2).sum())**(1/2))

    return d

In [ ]:
def errorMetrics(data, ref, colnames):
    
    # Initialize variables
    r = []
    r_pvalue = []
    mbe = []
    mpe = []
    mae = []
    mape = []
    rmse = []
    rmspe = []
    sam = []
    
    # Iterate over each point
    for col in colnames:
    
        # Calculate Pearson's correlation
        correlation = stats.pearsonr(data[col], ref[col])
    
        # Append correlation results to the lists
        r.append(correlation.statistic)
        r_pvalue.append(correlation.pvalue)
    
        # Calculate error
        error = data[col] - ref[col]
    
        # Calculate mean bias error
        mbe.append(error.mean())
    
        # Calculate mean percentage error
        mpe.append(error.mean() / ref[col].mean())
    
        # Calculate mean absolute error
        mae.append(abs(error).mean())
    
        # Calculate mean absolute percentage error
        mape.append(abs(error).mean() / ref[col].mean())
    
        # Calculate root mean square error (RMSE)
        rmse.append(np.sqrt((error ** 2).mean()))
    
        # Calculate root mean square percentage error
        rmspe.append(np.sqrt((error ** 2).mean()) / ref[col].mean())

        # Calculate spectral angle mapper
        sam.append(SAM(data[col], ref[col]))
    
    # Create dataframe
    error_metrics = pd.DataFrame({'point': data.columns[:-1],
                                  'r': r, 'pvalue': r_pvalue,
                                  'mbe': mbe, 'mpe': mpe,
                                  'mae': mae, 'mape': mape,
                                  'rmse': rmse, 'rmspe': rmspe,
                                  'sam': sam})
    
    return error_metrics